In [1]:
from elasticsearch import Elasticsearch
from openai import OpenAI


key = "my_awesome_key"
client = OpenAI(api_key = key)

es_client = Elasticsearch('http://localhost:9200')

#Q1
!curl localhost:9200

{
  "name" : "62d3b8f0d1af",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "0et4N5epSKa4winYWvZ7bA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [3]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/thorsten/projects/private_projects/llm_rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 303.64it/s]


In [4]:
# Q3
def q3(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    res = es_client.search(index=index_name, body=search_query)
       
    return res['hits']

In [5]:
query = "How do I execute a command in a running docker container?"
answer = q3(query)
answer

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'Ls8hcJABlFxxd84_LLS-',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'PM8hcJABlFxxd84_JrJI',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork

In [6]:
#Q4

def q4(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    res = es_client.search(index=index_name, body=search_query)
    return res['hits']


In [7]:
answer_q4 = q4(query)
answer_q4

{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'Ls8hcJABlFxxd84_LLS-',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'Tc8hcJABlFxxd84_LbQT',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use 

In [8]:
# Q5
def q5(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    docs = []
    res = es_client.search(index=index_name, body=search_query)
    for hit in res['hits']['hits']:
        docs.append(hit['_source'])
    return docs


    
    

In [14]:
answer_q5 = q5(query)

context = ""

for i in answer_q5:

    result = f"Q:{doc['question']}\nA:{doc['text']}\n\n"
    context += result

context = context.strip()


In [23]:
def build_prompt(query, search_results):
    prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query}

CONTEXT: 
{search_results}
""".strip()

    return prompt

In [24]:
chat = build_prompt(query, context)

In [26]:
len(chat)

1257

In [29]:
#q6
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

count_token = encoding.encode(chat)

len(count_token)


274